# Randomized Benchmarking: Simultaneous RB

In [ ]:
# Needs in terminal:
# $ quilc -S
# $ qvm -S

import numpy as np

from pyquil.api import get_benchmarker
from forest.benchmarking.randomized_benchmarking import *

%matplotlib inline

In [ ]:
from pyquil.api import get_qc, get_benchmarker
qc = get_qc("9q-square-noisy-qvm")
bm = get_benchmarker()

## Generate a 2 qubit RB experiment

In [ ]:
num_sequences = 10
depths = 3 * 2 ** np.arange(3, dtype=np.uint8)

expts = generate_rb_experiments(bm, [(0, 2)], depths, num_sequences)

## Run a single 2-qubit RB experiment on a QuantumComputer

In [ ]:
expt_results = acquire_rb_data(qc, expts, num_shots=100)

In [ ]:
expt_results[0]

## Generate and run a simultaneous rb experiment on qubit 1 and qubits (0, 2)

In [ ]:
qubits = [(1,), (0,2)]
simult_expts = generate_rb_experiments(bm, qubits, depths, num_sequences)

In [ ]:
simult_results = acquire_rb_data(qc, simult_expts, num_shots=100)

## Organize the expectations and std_errs by qubit group (1,) and (0,2)

In [ ]:
stats_by_group = get_stats_by_qubit_group(qubits, simult_results)
assert len(stats_by_group[(1,)]['expectation']) == len(depths) * num_sequences

## Fit the data separately for each group and plot.

In [ ]:
from forest.benchmarking.analysis.fitting import make_figure
group = qubits[0]
stats = stats_by_group[group]
exps = stats['expectation']
std_errs = stats['std_err']

fit = fit_rb_results([d for d in depths for _ in range(num_sequences)], exps, std_errs)
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")
rb_decay = fit.params['decay']

In [ ]:
group = qubits[1]
stats = stats_by_group[group]
exps = stats['expectation']
std_errs = stats['std_err']

fit = fit_rb_results([d for d in depths for _ in range(num_sequences)], exps, std_errs)
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")
rb_decay = fit.params['decay']

## Simultaneous unitarity

In [ ]:
num_sequences = 10
depths = 3 * 2 ** np.arange(3, dtype=np.uint8)
qubits = [(1,), (5,)]

uexpts = generate_unitarity_experiments(bm, qubits, depths, num_sequences)
print(uexpts[0])
simult_u_results = acquire_rb_data(qc, uexpts, num_shots=100)

In [ ]:
from forest.benchmarking.analysis.fitting import make_figure
stats_by_group = get_stats_by_qubit_group(qubits, simult_u_results)

group = qubits[0]
stats = stats_by_group[group]
exps = stats['expectation']
std_errs = stats['std_err']

fit = fit_unitarity_results([d for d in depths for _ in range(num_sequences)], exps, std_errs)
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")
unitarity = fit.params['decay'].value

In [ ]:
group = qubits[1]
stats = stats_by_group[group]
exps = stats['expectation']
std_errs = stats['std_err']

fit = fit_unitarity_results([d for d in depths for _ in range(num_sequences)], exps, std_errs)
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")
unitarity = fit.params['decay'].value